# multiple linear regression - 복잡한 코드의 예

In [5]:
# tensorflow version 1으로 입력
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

# data 입력
x1_data = [73,93,89,96,73]
x2_data = [80,88,91,98,66]
x3_data = [75,93,90,100,70]
y_data = [152, 185, 180, 196, 142]

# data 넣을 placeholder
x1 = tf.placeholder(tf.float32)
x2 = tf.placeholder(tf.float32)
x3 = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

# 학습시킬 변수와 bias 
w1 = tf.Variable(tf.random_normal([1]), name = 'weight1')
w2 = tf.Variable(tf.random_normal([1]), name = 'weight2')
w3 = tf.Variable(tf.random_normal([1]), name = 'weight3')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

# 학습에 필요한 다른 필수 요소들
# 1. hypothesis
hypothesis = x1 * w1 + x2 * w2 + x3 * w3 + b
# 2. cost
cost = tf.reduce_mean(tf.square(hypothesis-y))
# 3. optimizer & train
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-5)
train = optimizer.minimize(cost)

# Session start
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# learning start
for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train],
                                  feed_dict = {x1 : x1_data, x2 : x2_data, x3 : x3_data, y : y_data})
    if step % 100 == 0:
        print("cost : ", cost_val, ", hy_val : ", hy_val, ", train : ", train)
        
sess.close()

cost :  945.6367 , hy_val :  [125.6632  150.81361 148.77007 162.14984 114.71071] , train :  name: "GradientDescent_2"
op: "NoOp"
input: "^GradientDescent_2/update_weight1_2/ResourceApplyGradientDescent"
input: "^GradientDescent_2/update_weight2_2/ResourceApplyGradientDescent"
input: "^GradientDescent_2/update_weight3_2/ResourceApplyGradientDescent"
input: "^GradientDescent_2/update_bias_2/ResourceApplyGradientDescent"

cost :  2.1014454 , hy_val :  [152.86774 183.60487 181.03145 197.28458 139.74304] , train :  name: "GradientDescent_2"
op: "NoOp"
input: "^GradientDescent_2/update_weight1_2/ResourceApplyGradientDescent"
input: "^GradientDescent_2/update_weight2_2/ResourceApplyGradientDescent"
input: "^GradientDescent_2/update_weight3_2/ResourceApplyGradientDescent"
input: "^GradientDescent_2/update_bias_2/ResourceApplyGradientDescent"

cost :  2.0136323 , hy_val :  [152.8197  183.63806 181.01706 197.27165 139.7887 ] , train :  name: "GradientDescent_2"
op: "NoOp"
input: "^GradientDescen

cost :  1.0233294 , hy_val :  [152.14891 184.10239 180.81715 197.08298 140.43533] , train :  name: "GradientDescent_2"
op: "NoOp"
input: "^GradientDescent_2/update_weight1_2/ResourceApplyGradientDescent"
input: "^GradientDescent_2/update_weight2_2/ResourceApplyGradientDescent"
input: "^GradientDescent_2/update_weight3_2/ResourceApplyGradientDescent"
input: "^GradientDescent_2/update_bias_2/ResourceApplyGradientDescent"



##### 의문 1
왜 에러가 나는 것인지 모르겠다.
특히 tensorflow.compat.v1 으로 tf을 설정했으면 version 1코드는 다 작동이 잘 되어야 되는 게 아닌가...??

##### 의문 2
왜 input ~~ 이런 코드가 같이 나오는것인가...? 분명 그대로 입력했는데 알 수 없는 일이다ㅠㅠ

##### but 결론
이 코드는 변수가 많아질수록 복잡해져서 장기적으로 쓰길 권장하지 않는다.
=> x 데이터를 2차원(n차원) matrix로 만들고, y 데이터와 x 데이터만 사용하는 방법을 알아보자!

# multiple linear regression - 간단한 코드의 예

#### 여러 x와 y 변수를 matrix에 담아서 지정. 대문자로 X, Y라고 표현

In [35]:
tf.set_random_seed(777)

X_data = [[73., 80., 75.],
          [93., 88., 93.],
          [89., 91., 90.],
          [96., 98., 100.],
          [73., 66., 70.]]
Y_data = [[152.],
          [185.],
          [180.],
          [196.],
          [142.]]

#### placeholder로 마찬가지로 x, y data를 담을 placeholder 지정
다만 shape를 지정해야 함! b/c x, y의 형태가 matrix이기 때문!
& shape를 2차원으로 지정한다.
뒤의 element : 하나의 matrix에 몇 개의 요소가 들어가는지 => 3개.
앞의 element : 그런 matrix가 총 몇 개가 있는지 => 여기서는 5개이지만, 실제로는 무수히 많은 경우도 있다.
+ 그럴 경우는 None 이라고 표현!
** None : 없다 (X), 무수히 많다(=n)(O)

In [36]:
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

#### Variable로 학습시킬 변수를 지정
weight인 W과 bias인 b를 지정.
+ 이 때도 W, b에 shape를 설정해 줘야 한다. b/c 데이터 형태부터 matrix이므로 W, b도 matrix 형태의 변수여야 한다

In [38]:
W = tf.Variable(tf.random_normal([3, 1]), name = 'weight')
B = tf.Variable(tf.random_normal([1]), name = 'bias')

#### hypothesis 지정
+ matmul 이라는 메소드/함수를 사용한다!
아마도 matrix 연산을 하는 메소드인 듯

In [39]:
Hypothesis = tf.matmul(X, W) + B

#### cost 식 지정

In [40]:
Cost = tf.reduce_mean(tf.square(Hypothesis-Y))

#### Minimize 시키기

In [41]:
Optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-3)
Train = Optimizer.minimize(Cost)

#### Session 시작하기

In [42]:
sess2 = tf.Session()
sess2.run(tf.global_variables_initializer())

#### data로 모델 학습시키기

In [44]:
for step in range(2001):
    Cost_val, Hy_val, _ = sess2.run(
        [Cost, Hypothesis, Train], feed_dict = {X: X_data, Y: Y_data})
    if step % 100 == 0:
        print(step, "\nCost : ", Cost_val, "\nPrediction : ", Hy_val)
        
sess2.close()

0 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
100 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
200 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
300 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
400 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
500 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
600 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
700 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
800 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
900 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
1000 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
1100 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
1200 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
1300 
Cost :  nan 
Prediction :  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
1400 
Cost :  nan 
Prediction :  [[nan]
 [nan]

#### 후기
분명 코드를 그대로 입력했는데...ㅠㅠㅠㅠ 결과에 왜 nan이 뜨는지 알 수 없다!!! (납득불가)
이런 error 설명 모음집이라도 있었으면 좋겠다 흑흑